In [ ]:
# Setup environment

!apt-get -qq install xxd
!pip install pandas numpy matplotlib
!pip install tensorflow

In [ ]:
# Mounts Google Drive and copies a dataset folder (containing CSV files) locally

from google.colab import drive
drive.mount('/content/drive')

import os

# Path to the folder
folder_path = '/content/drive/My Drive/Colab Notebooks/dataset'

# List files in the folder
files = os.listdir(folder_path)
print(files)

!cp -r '/content/drive/My Drive/Colab Notebooks/dataset' './dataset'

# List files in the copied folder
local_files = os.listdir('./dataset')
print(local_files)

In [ ]:
# Plots acceleration and gyroscope data from CSV files

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

# Path to the folder containing the CSV files
directory = 'dataset'

# Get the list of CSV files in the directory
csv_files = [f for f in os.listdir(directory) if f.endswith('.csv')]

# Process each CSV file

# Accelerometer data
for filename in csv_files:
    print(f"Processing file (Acceleration): {filename}")

    # Load the data from the CSV file
    df = pd.read_csv(os.path.join(directory, filename))

    # Generate the index range
    index = range(1, len(df['aX']) + 1)

    # Plot Acceleration data
    plt.rcParams["figure.figsize"] = (20, 10)
    plt.plot(index, df['aX'], 'g.', label='x', linestyle='solid')
    plt.plot(index, df['aY'], 'b.', label='y', linestyle='solid')
    plt.plot(index, df['aZ'], 'r.', label='z', linestyle='solid')
    plt.title(f"Acceleration - {filename}")
    plt.xlabel("Sample #")
    plt.ylabel("Acceleration (mG)")
    plt.legend()
    plt.show()

# Gyroscope data
for filename in csv_files:
    print(f"Processing file (Gyroscope): {filename}")

    # Load the data from the CSV file
    df = pd.read_csv(os.path.join(directory, filename))

    # Generate the index range
    index = range(1, len(df['gX']) + 1)

    # Plot Acceleration data
    plt.rcParams["figure.figsize"] = (20, 10)
    plt.plot(index, df['gX'], 'g.', label='x', linestyle='solid')
    plt.plot(index, df['gY'], 'b.', label='y', linestyle='solid')
    plt.plot(index, df['gZ'], 'r.', label='z', linestyle='solid')
    plt.title(f"Gyroscope - {filename}")
    plt.xlabel("Sample #")
    plt.ylabel("Gyroscope (dps)")
    plt.legend()
    plt.show()

In [ ]:
# Loads and processes gesture data from CSV files, normalizes the sensor values,
# and prepares a dataset with input features and one-hot encoded labels for each gesture.

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# the list of gestures that data is available for
GESTURES = [
    "wing", #W
    "ring", #O
    "slope", #L
    "nogesture"
]

# the number of samples per gesture
SAMPLES_PER_GESTURE = 100

NUM_GESTURES = len(GESTURES)

# create a one-hot encoded matrix that is used in the output
ONE_HOT_ENCODED_GESTURES = np.eye(NUM_GESTURES)

inputs = []
outputs = []

# read each csv file and push an input and output
for gesture_index in range(NUM_GESTURES):
  gesture = GESTURES[gesture_index]
  print(f"Processing index {gesture_index} for gesture '{gesture}'.")

  output = ONE_HOT_ENCODED_GESTURES[gesture_index]

  df = pd.read_csv("dataset/" + gesture + ".csv")

  # calculate the number of gesture recordings in the file
  num_recordings = int(df.shape[0] / SAMPLES_PER_GESTURE)

  for i in range(num_recordings):
    tensor = []
    for j in range(SAMPLES_PER_GESTURE):
      index = i * SAMPLES_PER_GESTURE + j
      tensor += [
          (df['aX'][index]+2000)/4000,
          (df['aY'][index]+2000)/4000,
          (df['aZ'][index]+2000)/4000,
          (df['gX'][index]+2000)/4000,
          (df['gY'][index]+2000)/4000,
          (df['gZ'][index]+2000)/4000
      ]

    inputs.append(tensor)
    outputs.append(output)

# convert the list to numpy array
inputs = np.array(inputs)
outputs = np.array(outputs)

print("\nDataset parsing and preparation complete.")
print(f"Shape of inputs: {inputs.shape}")
print(f"Shape of outputs: {outputs.shape}")

In [ ]:
# Set a fixed random seed value, for reproducibility, this will allow us to get the same random numbers each time the notebook is run

import tensorflow as tf

SEED = 1337
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Randomize the order of the inputs, so they can be evenly distributed for training, testing, and validation
num_inputs = len(inputs)
randomize = np.arange(num_inputs)
np.random.shuffle(randomize)

# Swap the consecutive indexes (0, 1, 2, etc) with the randomized indexes
inputs = inputs[randomize]
outputs = outputs[randomize]

# Split the dataset into three sets: training 60%, testing 20% and validation 20%
TRAIN_SPLIT = int(0.6 * num_inputs)
TEST_SPLIT = int(0.2 * num_inputs + TRAIN_SPLIT)

inputs_train, inputs_test, inputs_validate = np.split(inputs, [TRAIN_SPLIT, TEST_SPLIT])
outputs_train, outputs_test, outputs_validate = np.split(outputs, [TRAIN_SPLIT, TEST_SPLIT])

print("Data set randomization and splitting complete.")
print(f"Shape of inputs train: {inputs_train.shape}")
print(f"Shape of outputs train: {outputs_train.shape}")
print(f"Shape of inputs test: {inputs_test.shape}")
print(f"Shape of outputs test: {outputs_test.shape}")
print(f"Shape of inputs validate: {inputs_validate.shape}")
print(f"Shape of outputs validate: {outputs_validate.shape}")

In [ ]:
# Build the model and train it

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(50, activation='relu'))
model.add(tf.keras.layers.Dense(15, activation='relu'))
model.add(tf.keras.layers.Dense(NUM_GESTURES, activation='softmax'))  # Output layer with softmax activation, one unit per gesture (multi-class classification).

# RMSProp adapts the learning rate, stabilizing training for gesture recognition.
# Categorical cross-entropy is used for multi-class classification with one-hot encoded labels.
# Accuracy tracks how many gestures are predicted correctly.
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model for 600 epochs with a batch size of 8.
# Epochs: 600 means the model will train for 600 iterations over the entire dataset; increasing the number can improve accuracy, but it may also increase training time.
# Batch size: 8 means the model processes 8 samples at a time before updating weights; smaller batch sizes can improve generalization but may increase training time.
history = model.fit(inputs_train, outputs_train, epochs=600, batch_size=8, validation_data=(inputs_validate, outputs_validate))

In [ ]:
# Visualizes the training and validation loss and accuracy to help monitor the model's performance during training.
# The plots help check if the model is improving and whether it is overfitting or underfitting.

import matplotlib.pyplot as plt

# Get training and validation loss and accuracy
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

# Plot loss curves
plt.figure(figsize=(12, 6))

# Loss plot
plt.subplot(1, 2, 1)
plt.plot(train_loss, label='Training Loss', color='blue')
plt.plot(val_loss, label='Validation Loss', color='red')
plt.title('Loss Curve')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Accuracy plot
plt.subplot(1, 2, 2)
plt.plot(train_accuracy, label='Training Accuracy', color='blue')
plt.plot(val_accuracy, label='Validation Accuracy', color='red')
plt.title('Accuracy Curve')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Show the plots
plt.tight_layout()
plt.show()

In [ ]:
# use the model to predict the test inputs
predictions = model.predict(inputs_test)

# print the predictions and the expected ouputs
print("predictions =\n", np.round(predictions, decimals=3))
print("actual =\n", outputs_test)

# Verifica se c'è una grande differenza tra predizioni e valori effettivi
differences = np.abs(np.round(predictions, decimals=3) - outputs_test)
print("Differences =\n", differences)

threshold = 0.1  # Definisci una soglia di errore accettabile
is_good_prediction = differences <= threshold
print("Good predictions (within threshold) =\n", is_good_prediction)

anomaly_threshold = 0.5  # Definisci una soglia per identificare anomalie
anomalies = differences > anomaly_threshold
print("Anomalies detected:\n", anomalies)

correct_percentage = np.mean(differences < threshold) * 100
print(f"Percentage of good predictions (within threshold): {correct_percentage}%")

In [ ]:
# Convert the model in TensorFlow Lite format

def representative_dataset_gen():
    for i in range(len(inputs_train)):  # Iterate over the entire training dataset
        # Take a sample from the training data and convert it to float32 format
        yield [inputs_train[i].astype(np.float32)]  # Ensure the data is in float32 format

# Create the TFLite converter from the trained Keras model
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Set optimizations for the model
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Set the representative dataset for calibration (needed for optimizations)
converter.representative_dataset = representative_dataset_gen

# Set the target specification for supported operations
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

# Set input and output types for inference to float32 (no quantization)
converter.inference_input_type = tf.float32
converter.inference_output_type = tf.float32

# Convert the model to TensorFlow Lite format
tflite_model = converter.convert()

# Save the converted model to disk
open("gesture_model.tflite", "wb").write(tflite_model)

# Get the model size and print it
basic_model_size = os.path.getsize("gesture_model.tflite")
print("Model size: %d bytes" % basic_model_size)

In [ ]:
# Displays this information related to TFLite model to help understand the model's expected input/output format.

# Load the TensorFlow Lite model
interpreter = tf.lite.Interpreter(model_path="gesture_model.tflite")

# Get the number of inputs and outputs
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Show the number of inputs and outputs, and their shapes
print(f"Number of inputs: {len(input_details)}")
print(f"Number of outputs: {len(output_details)}")

# Show the details of the inputs
for i, input_detail in enumerate(input_details):
    print(f"Input {i}:")
    print(f"  Type: {input_detail['dtype']}")
    print(f"  Shape: {input_detail['shape']}")
    print(f"  Number of dimensions: {len(input_detail['shape'])}")

# Show the details of the outputs
for i, output_detail in enumerate(output_details):
    print(f"Output {i}:")
    print(f"  Type: {output_detail['dtype']}")
    print(f"  Shape: {output_detail['shape']}")

In [ ]:
# Demonstrates how to use a TFLite interpreter to process input data and obtain gesture recognition results.

import numpy as np
import tensorflow as tf

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="gesture_model.tflite")
interpreter.allocate_tensors()

# Get the input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Show the input and output details
print("Input details:", input_details)
print("Output details:", output_details)

# Prepare the input data
# Required shape: [1, 270], Type: float32
input_shape = input_details[0]['shape']  # Get the input shape

# Assume inputs_test is already defined as a NumPy array
# Select a vector from inputs_test (e.g., the first vector)
input_data = inputs_test[4]  # Select the first example (modify the index for another example)
input_data = np.expand_dims(input_data, axis=0)  # Add batch dimension to make it [1, 270]
input_data = input_data.astype(input_details[0]['dtype'])  # Ensure the type is float32

# Set the input data to the model
interpreter.set_tensor(input_details[0]['index'], input_data)

# Perform inference
interpreter.invoke()

# Get the results
output_data = interpreter.get_tensor(output_details[0]['index'])

# Print the results
print("Input data:", input_data)
print("Inference result:", output_data)

In [ ]:
# Convert TFLite file in .c file

!echo "// Auto-generated serialization of TFLite flatbuffers" > /content/sl_tflite_micro_model.c
!echo "" >> /content/sl_tflite_micro_model.c
!echo "#include \"em_device.h\"" >> /content/sl_tflite_micro_model.c
!echo "#include \"sl_tflite_micro_model.h\"" >> /content/sl_tflite_micro_model.c
!echo "" >> /content/sl_tflite_micro_model.c
!echo "// Model data generated from .tflite file" >> /content/sl_tflite_micro_model.c
!echo "const uint8_t sl_tflite_model_array[] __ALIGNED(4) = {" >> /content/sl_tflite_micro_model.c
!cat gesture_model.tflite | xxd -i      >> /content/sl_tflite_micro_model.c
!echo "};"                              >> /content/sl_tflite_micro_model.c
model_size = os.path.getsize("gesture_model.tflite")
!echo "const uint32_t sl_tflite_model_len = {model_size}UL;" >> /content/sl_tflite_micro_model.c

print("Model size: %d bytes" % model_size)
model_h_size = os.path.getsize("sl_tflite_micro_model.c")
print(f"sl_tflite_micro_model.c size: {model_h_size:,} bytes.")